In [ ]:
%load_ext lab_black

import numpy as np
import pandas as pd
from PIL import Image

from torchvision import transforms
from tqdm import tqdm_notebook as tqdm
import cufflinks

cufflinks.go_offline(False)

ETA = 2.1
BETA = 1

## Import image

In [ ]:
impath = "../images/markov.jpg"

im = Image.open(impath)
im = transforms.Resize(256)(
    transforms.RandomHorizontalFlip(1)(
        transforms.RandomVerticalFlip(1)(transforms.Grayscale()(im))
    )
)

im_tensor = transforms.ToTensor()(im)
im_array = im_tensor.numpy().reshape(im_tensor.shape[1], im_tensor.shape[2])
y = np.where(im_array > 0.4, np.float32(1), np.float32(-1))

In [ ]:
def display_image(image):
    return transforms.ToPILImage()(transforms.ToTensor()(image))


display_image(y)

## Noise pollution!

In [ ]:
for row in tqdm(range(y.shape[0])):
    for col in range(y.shape[1]):
        switch = np.random.binomial(1, 0.1)
        if switch:
            y[row, col] *= -1

x = y.copy()

In [ ]:
transforms.ToPILImage()(transforms.ToTensor()(y))

## Function to calculate total energy

In [ ]:
def calculate_total_energy(x, y):
    energy = 0
    energy += -ETA * (x * y).sum()
    energy += -BETA * (x[:-1, :] * x[1:, :]).sum()
    energy += -BETA * (x[:, :-1] * x[:, 1:]).sum()
    return energy

## Function to set state of pixel according to lowest energy configuration

In [ ]:
def set_best_state(x, y, row, col):
    energies = []
    STATES = [-1, 1]
    for state in STATES:
        x[row, col] = state
        energies.append(calculate_total_energy(x, y))
    best_state = STATES[np.argmin(energies)]
    x[row, col] = best_state

## Loop through pixels, setting each to their best state

In [ ]:
energies = [calculate_total_energy(x, y)]
num_rounds = 0
delta_energy = 1
while delta_energy > 1e-8:
    for row in tqdm(range(x.shape[0])):
        for col in range(x.shape[1]):
            set_best_state(x, y, row, col)
    energies.append(calculate_total_energy(x, y))
    delta_energy = energies[-2] - energies[-1]
    num_rounds += 1
    if num_rounds == 100:
        break
    if num_rounds % 10 == 0:
        print(num_rounds)

## Visualise loss!

In [ ]:
pd.DataFrame({"energy": energies}).reset_index().rename(
    {"index": "n_epoch"}, axis=1
).set_index("n_epoch").iplot(xTitle="Epoch number", yTitle="Energy")

## Reminder: here was the observed image...

In [ ]:
display_image(y)

## ...and here's the reconstructed image!

In [ ]:
display_image(x)